### Football Data Aggregator
aggregates football results by 2 weeks, month, year

In [24]:
import pandas as pd
import numpy as np

In [25]:
rc = pd.read_csv('results_clean.csv', parse_dates=["date"])
rc.drop(["city", "country"],inplace=True, axis=1)
rc.head()

,date,home_team,away_team,home_score,away_score,tournament,neutral,winner,winner_loc
0,2010-03-03,Austria,Denmark,2,1,Friendly,False,Austria,Home
1,2010-03-03,Belgium,Croatia,0,1,Friendly,False,Croatia,Away
2,2010-03-03,France,Spain,0,2,Friendly,False,Spain,Away
3,2010-03-03,Malta,Finland,1,2,Friendly,False,Finland,Away
4,2010-03-03,Poland,Bulgaria,2,0,Friendly,False,Poland,Home


In [26]:
rc["home_win"] = rc["home_team"] == rc["winner"]
rc["away_win"] = rc["away_team"] == rc["winner"]
rc.head()

,date,home_team,away_team,home_score,away_score,tournament,neutral,winner,winner_loc,home_win,away_win
0,2010-03-03,Austria,Denmark,2,1,Friendly,False,Austria,Home,True,False
1,2010-03-03,Belgium,Croatia,0,1,Friendly,False,Croatia,Away,False,True
2,2010-03-03,France,Spain,0,2,Friendly,False,Spain,Away,False,True
3,2010-03-03,Malta,Finland,1,2,Friendly,False,Finland,Away,False,True
4,2010-03-03,Poland,Bulgaria,2,0,Friendly,False,Poland,Home,True,False


In [27]:
# Change this function to change time slice
rc["year_month"] = rc["date"].apply(lambda x: str(x.year) +"-"+ "{:02d}".format(x.month))
rc["games_played"] = 1

home = rc.groupby(["home_team", "year_month"]).sum().add_suffix('').reset_index()
away = rc.groupby(["away_team", "year_month"]).sum().add_suffix('').reset_index()
vict = rc.groupby(["winner", "year_month"]).sum().add_suffix('').reset_index()

comb = home.rename(columns={"home_team":"team"}).append(away.rename(columns={"away_team":"team"}))
comb = comb.groupby(["team", "year_month"]).sum().add_suffix('').reset_index()
comb

,team,year_month,home_score,away_score,neutral,home_win,away_win,games_played
0,Austria,2010,7,8,0,1,2,4
1,Austria,2011,14,10,0,3,3,6
2,Austria,2012,4,3,0,1,1,3
3,Austria,2013,7,4,0,3,1,4
4,Austria,2014,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...
291,Sweden,2017,18,6,0,5,1,7
292,Sweden,2018,7,4,3,3,1,7
293,Sweden,2019,10,8,1,4,2,7
294,Sweden,2020,13,7,0,5,2,7


In [28]:
comb.to_csv("results_month.csv")